In [1]:
import duckdb
import polars as pl
import lancedb

/home/evan/Documents/hypersync_lancedb/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cd ../

/home/evan/Documents/hypersync_lancedb


/home/evan/Documents/hypersync_lancedb/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### duckb read lance table

In [4]:
db: lancedb.DBConnection = lancedb.connect("data")
# open the table as a lance dataset to make it accessible for duckdb
lance_dataset_table: lancedb.table = db.open_table("transactions").to_lance()

In [5]:
txs_table: lancedb.table = db.open_table("transactions").to_lance()

In [6]:
duckdb.sql("SELECT MAX(block_number) FROM txs_table")

┌───────────────────┐
│ max(block_number) │
│      uint64       │
├───────────────────┤
│          19855000 │
└───────────────────┘

In [7]:
duckdb.sql("SELECT COUNT(DISTINCT block_number) FROM txs_table")

┌──────────────────────────────┐
│ count(DISTINCT block_number) │
│            int64             │
├──────────────────────────────┤
│                        54973 │
└──────────────────────────────┘

### polars read lance table

In [3]:
table_name = "blocks"
db: lancedb.DBConnection = lancedb.connect("data")

In [4]:
# Try to open and merge data into existing table.
blocks_table: lancedb.table = db.open_table(table_name)

blocks_index: pl.DataFrame = (
    blocks_table.to_polars()
    .select("block_number")
    .sort(by="block_number", descending=True)
    .collect()["block_number"]
)

In [5]:
blocks_index.shape

(55002,)

In [6]:
txs_table: lancedb.table = db.open_table("transactions")
txs_index: pl.DataFrame = (
    txs_table.to_polars()
    .sort(by="block_number", descending=True)
    .drop("input", "logs_bloom", "sighash")
    .collect()
)

In [7]:
txs_index.select("block_number").unique().shape

(54973, 1)

### Hypersync parquet read

Verifies that the data is the same

In [3]:
blocks_lf = pl.scan_parquet("data/blocks.parquet")
txs_lf = pl.scan_parquet("data/transactions.parquet")

In [4]:
blocks_df = blocks_lf.sort(by="number", descending=True).collect()

In [5]:
blocks_df.shape

(55002, 19)

In [6]:
txs_df = (
    txs_lf.sort(by="block_number", descending=True)
    .drop("input", "logs_bloom", "sighash")
    .collect()
)

In [7]:
txs_df.select("block_number").unique().shape

(54973, 1)